In [215]:
import os
import requests
import googlemaps
import json
from gomaps import maps_search
import time
import googlemaps
import pandas as pd
import pgeocode
import requests
import json
import re
import numpy as np
from cleantext import clean

Since the GPL-licensed package `unidecode` is not installed, using Python's `unicodedata` package which yields worse results.


In [3]:
#Function to extract place_id of user's google maps link
def get_location_from_url(url):
    # Regular expression to match latitude and longitude
    regex = r"@(-?\d+\.\d+),(-?\d+\.\d+)"
    
    # Search for latitude and longitude in the URL
    match = re.search(regex, url)
    
    if match:
        # Extract latitude and longitude from the regex match
        latitude = float(match.group(1))
        longitude = float(match.group(2))
        
        return (latitude, longitude)
    else:
        return None
    

In [4]:
def get_place_id(latitude, longitude, api_key):
    # Set up the API endpoint URL and parameters
    url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    params = {
        "location": f"{latitude},{longitude}",
        "rankby": "distance",
        "category": "restaurant",
        "key": api_key
    }
    
    # Send the API request and get the JSON response
    response = requests.get(url, params=params).json()
    
    # Check if any results were returned
    if len(response["results"]) > 0:
        # Extract the place ID from the first result
        place_id = response["results"][0]["place_id"]
        return place_id
    else:
        print("Error in get_place_id")

In [5]:
def extract_place_details(place_id, api_key):
    url = f'https://maps.googleapis.com/maps/api/place/details/json?place_id={place_id}&key={api_key}'

    # Send a GET request to the API endpoint and retrieve the response
    response = requests.get(url)

    # Parse the JSON response and convert it to a dictionary
    result = json.loads(response.text)

    try:
        return [result["result"]]
    
    except:
        print("Error in extract_place_details")

In [6]:
def transform_to_df(place_details):
    user_input_details = pd.DataFrame(data = place_details)
    return user_input_details

In [15]:
#Option 1: get user details based on google maps link
def process_user_input_details1():
    user_input = input("please insert the goolge maps link of your favorite location: ")
    api_key = open("/Users/lukasbauerschmidt/Desktop/Ironhack/5. Final Project/Final-Project-Ironhack/2. Google API Key/key.txt").read()
    latitude, longitude = get_location_from_url(user_input)
    place_id = get_place_id(latitude, longitude, api_key)
    place_details = extract_place_details(place_id, api_key)
    user_input_details = transform_to_df(place_details)
    
    return user_input_details

In [16]:
#Option 2: get user details based on place_id searched in the following website "https://developers.google.com/maps/documentation/javascript/examples/places-placeid-finder"
def process_user_input_details2():
    print("1. Please find your place_id of your favorite location using the following link: https://developers.google.com/maps/documentation/javascript/examples/places-placeid-finder")
    place_id = input("please insert the place_id of your desired location: ")
    api_key = open("/Users/lukasbauerschmidt/Desktop/Ironhack/5. Final Project/Final-Project-Ironhack/2. Google API Key/key.txt").read()
    place_details = extract_place_details(place_id, api_key)
    user_input_details = transform_to_df(place_details)
    
    return user_input_details

In [9]:
def get_coordinates_from_details(user_input_details):
    latitude = round(user_input_details["geometry"][0]["location"]["lat"], 7)
    longitude = round(user_input_details["geometry"][0]["location"]["lng"], 7)
    return latitude, longitude

In [35]:
def process_user_input_high_level(user_input_details):
    latitude, longitude = get_coordinates_from_details(user_input_details)
    
    #Google Maps API parameters
    search_string = 'restaurant'
    distance = 0.0001
    rankby=distance
    
    api_key = open("/Users/lukasbauerschmidt/Desktop/Ironhack/5. Final Project/Final-Project-Ironhack/2. Google API Key/key.txt").read()
    map_client = googlemaps.Client(api_key)
    response = map_client.places_nearby(location=(latitude, longitude), keyword=search_string, radius=distance)
    
    result = response.get('results')

    for i, business in enumerate(result):
        latitude_hl = business["geometry"]["location"]["lat"] 
        longitude_hl = business["geometry"]["location"]["lng"]
        
        if latitude_hl == latitude and longitude_hl == longitude:
            high_level_result = transform_to_df([result[i]])

            high_level_result['url'] = 'https://www.google.com/maps/place/?q=place_id:' + high_level_result['place_id']
            
            return high_level_result
        
        else:
            print("Error in process_user_input_high_level")

In [79]:
def concat(df_details, df_high_level):
    final = pd.concat([df_high_level.set_index("place_id"), df_details.set_index("place_id")], axis = 1, join="inner")
    return final

In [114]:
def add_missing_cols(final_df):
    benchmark_df = pd.read_csv("/Users/lukasbauerschmidt/Desktop/Ironhack/5. Final Project/Leo Repository/df_with_model_input.csv")
    benchmark_cols = benchmark_df.columns
    final_cols = final_df.columns
    
    for benchmark_column in benchmark_cols:
        if benchmark_column not in final_cols:
            final_df[benchmark_column] = np.nan
        else:
            pass
        
    return final_df

In [220]:
def clean_data(final_df):
    #dropping useless columns
    
    final_df = final_df.loc[:, ~final_df.columns.duplicated(keep='first')]
    
    cols_to_drop = ['Unnamed: 0', 'secondary_opening_hours', 'utc_offset', 'current_opening_hours', 'adr_address', 'address_components', 'icon', 'icon_background_color', 'icon',
       'icon_mask_base_uri', 'scope', 'formatted_phone_number', "opening_hours"]
    
    final_df = final_df.drop(cols_to_drop, axis=1)
    
    final_df = final_df[['name', 'reference', 'geometry', 'formatted_address',
       'price_level', 'rating', 'user_ratings_total', 'types','editorial_summary', 'reviews',
       'curbside_pickup', 'delivery', 'dine_in', 'takeout', 'reservable',
       'serves_beer', 'serves_breakfast', 'serves_brunch', 'serves_dinner', 'serves_lunch', 'serves_vegetarian_food', 'serves_wine',
       'wheelchair_accessible_entrance', 'website', 'international_phone_number', 'photos', 'vicinity', 'url', 'business_status','permanently_closed', 'plus_code']]
    
    
    review_dct = {}
    for i, item in enumerate(final_df["reviews"][0]):
        review_dct[f"review{i}"] = clean(item["text"], no_emoji=True, no_line_breaks=True)

    try:
        review_dct.update({"editorial_summary" : ast.literal_eval(final_df["editorial_summary"][0])["overview"]})
        
    except:
        review_dct.update({"editorial_summary" : ""})

    try:
        review_dct.update({"types" : ast.literal_eval(final_df["types"][0].replace("'point_of_interest', ", ""))})
        
    except:
        review_dct.update({"types" : ""})
    
    final_df["model_input"] = str(review_dct).replace("{", "").replace("}","")
    
    #final_df["model_input"].apply(lambda x: str(x).replace("'review0':" , "").replace("'review1':" , "").replace("'review2':" , "").replace("'review3':" , "").replace("'review4':" , "").replace("'types':" , "").replace("'editorial_summary':" , ""))  
    
    return final_df

In [231]:
def user_input():
    user_input_details = process_user_input_details2()
    user_input_high_level = process_user_input_high_level(user_input_details)
    final = concat(user_input_details, user_input_high_level)
    final = add_missing_cols(final)
    final_cleaned = clean_data(final)
    
    return final_cleaned

In [236]:
user_input().columns

1. Please find your place_id of your favorite location using the following link: https://developers.google.com/maps/documentation/javascript/examples/places-placeid-finder
please insert the place_id of your desired location: ChIJywdI5etRqEcRUVD-ycuGL6M


Index(['name', 'reference', 'geometry', 'formatted_address', 'price_level',
       'rating', 'user_ratings_total', 'types', 'editorial_summary', 'reviews',
       'curbside_pickup', 'delivery', 'dine_in', 'takeout', 'reservable',
       'serves_beer', 'serves_breakfast', 'serves_brunch', 'serves_dinner',
       'serves_lunch', 'serves_vegetarian_food', 'serves_wine',
       'wheelchair_accessible_entrance', 'website',
       'international_phone_number', 'photos', 'vicinity', 'url',
       'business_status', 'permanently_closed', 'plus_code', 'model_input'],
      dtype='object')